In [1]:
from pathlib import Path

from chic.structure import Structure

---
### ZIF-8

In [2]:
# path to test example directory.
eg_dir = Path('examples/cg_zif8')

# instantiate a Structure object from a CIF file.
zif8 = Structure.from_cif(eg_dir / 'ZIF-8-sod.cif', cores=4, verbose=True)

# we can apply methods native to the Pymatgen Structure class.
zif8.remove_species('O')

# we can apply methods native to the CHIC Structure class.
zif8.average_element_pairs('H', rmin=0.0, rmax=0.9)

# a helper function for computing the neighbour list using CrystalNN. This is 
# the bottleneck in the code, and is where Python's multiprocessing module
# is invoked.
zif8.get_neighbours_crystalnn()

# determine atomic clusters.
zif8.find_atomic_clusters()

# coarse-grain with centroid method.
zif8.get_coarse_grained_net()

# we can check the coarse-grained net seems reasonable.
zif8.overlay_cg_atomistic_representations(eg_dir / 'ZIF-8-sod-overlay.cif')

# we can write the coarse-grained net to a CIF file with the bonds formatted
# for TopoCIF.
zif8.to_cif(eg_dir / 'ZIF-8-sod-cg.cif', write_bonds=True, name='ZIF-8-cg')

# we can also write to LAMMPS data file. It is more obvious if the bonds are
# not correct because we can view it directly in OVITO.
zif8.to_lammps_data(
    eg_dir / 'ZIF-8-sod-cg.data', write_bonds=True, name='ZIF-8-cg'
)

average_element_pairs() took 0.02 seconds to execute.
get_neighbours_crystalnn() took 13.44 seconds to execute.
find_atomic_clusters() took 0.04 seconds to execute.
get_coarse_grained_net() took 0.01 seconds to execute.


In [3]:
# if you have crystal toolkit installed, you can visualise the overlay within
# the notebook (note this can take some time to load).
zif8.overlay_cg_atomistic_representations(crystal_toolkit=True)

No module named 'phonopy'


In [4]:
from chic.utils import crystal_toolkit_display

# we can also visualise individual atomic clusters. For example, let's look at
# the first 'b-type' atomic cluster in the ZIF-8 framework.
molecule = zif8._atomic_clusters[('b', 1)].to_molecule()

# center it and view it in crystal toolkit.
crystal_toolkit_display(molecule.get_centered_molecule())

---
### ZIF-4

In [9]:
# path to test example directory.
eg_dir = Path('examples/cg_zif4')

# instantiate a Structure object from a CIF file. we turn off the timings
# this time (verbose=False).
zif4 = Structure.from_cif(eg_dir / 'ZIF-4-cag.cif', cores=4, verbose=False)

# here the thermal motion of the imidazolate ring leads to split positions, so
# we average the positions of the atoms.
zif4.average_element_pairs('H', rmin=0.99, rmax=1.1)
zif4.average_element_pairs('C', rmin=0.6, rmax=0.7)

# use Pymatgen to write the "tidy" structure to a CIF. note we need to convert
# the Path object to a string for the Pymatgen writer function.
zif4.to(str(eg_dir / 'ZIF-4-cag-tidy.cif'));

In [10]:
# follow coarse-graining protocol.
zif4.get_neighbours_crystalnn()
zif4.find_atomic_clusters()
zif4.get_coarse_grained_net()
zif4.overlay_cg_atomistic_representations(eg_dir / 'ZIF-4-cag-overlay.cif')

# we can write the coarse-grained net to a CIF file with the bonds formatted
# for TopoCIF.
zif4.to_cif(eg_dir / 'ZIF-4-cag-topocif.cif', write_bonds=True, name='ZIF-4-cg')

# we can also write to LAMMPS data file. It is more obvious if the bonds are
# not correct because we can view it directly in OVITO.
zif4.to_lammps_data(
    eg_dir / 'ZIF-4-cag-cg.data', write_bonds=True, name='ZIF-4-cg'
)

---
### From LAMMPS data.

In this case, we make use of the LAMMPS data information to determine the atomic
clusters. Here, we have the ZIF-4 structure with each molecule labelled with
a unique molecule ID. Note, for this to work, even single atoms (e.g. zinc 
atoms) should have a unique molecule ID.

We pass the kwarg `cluster_by_molecule_id=True` to identify the clusters using
the molecule IDs.

In [ ]:
vejyuf = Structure.from_lammps_data(
    'examples/VEJYUF.data', 
    cluster_by_molecule_id=True
)

# now call the coarse-graining methods as usual.
vejyuf.get_coarse_grained_net()

# and visualise the coarse-grained structure in the overlay mode.
vejyuf.overlay_cg_atomistic_representations(
    'examples/vejyuf.cif'
)

# write to CIF. note, since we haven't assigned any inter-molecular bonds,
# the CIF will not contain any bonding information.
vejyuf.to_cif(
    'examples/vejyuf-cg.cif', 
    name='ZIF-4-cg-from-lammps'
)

In [ ]:
from pymatgen.io.lammps.outputs import parse_lammps_dumps

In [ ]:
zif4_400K = Structure.from_lammps_data(
    'examples/lammps_dump/melt.plateau_400K.data', 
    cluster_by_molecule_id=True
)

zif4_400K.get_coarse_grained_net()

zif4_400K.overlay_cg_atomistic_representations(
    'examples/lammps_dump/overlay.cif'
)


In [ ]:
trajectory = parse_lammps_dumps(
    'examples/lammps_dump/plateau_400K.dump'
)

In [ ]:
len(trajectory)

In [ ]:
first_dump = next(trajectory)

In [ ]:
first_dump.as_dict()

In [ ]:
import freud

In [ ]:
box = freud.box.Box.from_matrix(zif8.lattice.matrix)
voro = freud.locality.Voronoi()
voro.compute((box, zif8.cart_coords))

In [ ]:
voro.face_areas

In [ ]:
from pymatgen.analysis.local_env import VoronoiNN

In [ ]:
def adjust_weight(neighbour_info):
    neighbour_info["weight"] *= neighbour_info["poly_info"]["solid_angle"] / \
        neighbour_info["poly_info"]["area"]


def custom_crystalNN(
    structure,
    weighted_cn=False,
    cation_anion=False,
    distance_cutoffs=(0.5, 1),
    x_diff_weight=3.0,
    porous_adjustment=True,
    search_cutoff=7,
    fingerprint_length=None
):
    """
    """

    # Get the VoronoiNN list of neighbours for each site.
    vnn = VoronoiNN(weight="solid_angle", cutoff=search_cutoff)
    all_nn = vnn.get_all_nn_info(structure)

    print(len(all_nn), len(zif8))
    print(all_nn)

    # iterate through each site.
    for site_index, site_neighbours in enumerate(all_nn):
        for neighbour_index, neighbour_info in site_neighbours.items():
            print(neighbour_info)
            if porous_adjustment:
                adjust_weight(neighbour_info)

        #print()


custom_crystalNN(zif8)

In [ ]:
vnn = VoronoiNN(weight="solid_angle", cutoff=15)

polyhedra = vnn.get_all_voronoi_polyhedra(zif8)

In [ ]:
cnn = CrystalNN(    weighted_cn=True, cation_anion=False,
                    distance_cutoffs=(0.5,1), x_diff_weight=0,
                    porous_adjustment=True, search_cutoff=15,
                    fingerprint_length=None
)

nlist_crystalNN = {i: cnn.get_nn_data(zif8, i) for i in range(len(zif8))}

In [ ]:
nlist_freud = create_bond_dict(voro.nlist[:])

In [ ]:
sites = [zif8[ix] for ix in set(nlist_freud[0])]
all_vertices = voro.polytopes[0]
center_coordinates = zif8[0].coords

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 10))
scatter = ax.scatter(*center_coordinates[:2], c='k')
scatter = ax.scatter(*all_vertices[:2].T, c='k')
plt.show()

In [ ]:
nlist_crystalNN[0]

In [ ]:
vnn = VoronoiNN(weight="solid_angle", cutoff=12)
nn = vnn.get_nn_info(zif8, 0)
nn

In [ ]:
def hexagonal_lattice(rows=3, cols=3, noise=0, seed=None):
    if seed is not None:
        np.random.seed(seed)
    # Assemble a hexagonal lattice
    points = []
    for row in range(rows * 2):
        for col in range(cols):
            x = (col + (0.5 * (row % 2))) * np.sqrt(3)
            y = row * 0.5
            points.append((x, y, 0))
    points = np.asarray(points)
    points += np.random.multivariate_normal(
        mean=np.zeros(3), cov=np.eye(3) * noise, size=points.shape[0]
    )
    # Set z=0 again for all points after adding Gaussian noise
    points[:, 2] = 0

    # Wrap the points into the box
    box = freud.box.Box(Lx=cols * np.sqrt(3), Ly=rows, is2D=True)
    points = box.wrap(points)
    return box, points

box, points = hexagonal_lattice(rows=12, cols=8, noise=0.03, seed=2)
voro = freud.locality.Voronoi()
voro.compute((box, points))

# Plot Voronoi with points and a custom cmap
plt.figure()
ax = plt.gca()
voro.plot(ax=ax, cmap="RdBu")
ax.scatter(points[:, 0], points[:, 1], s=2, c="k")
plt.show()

In [ ]:
box = freud.box.Box.from_matrix(zif8.lattice.matrix)
voro = freud.locality.Voronoi()
voro.compute((box, zif8.cart_coords))
voronoi_neighbours = create_bond_dict(voro.nlist[:])

In [ ]:
def shared_coordinates(arr1, arr2, rounding=4):
    """
    Return the coordinates of the elements that are shared between two arrays.
    """

    a1 = np.round(arr1, rounding)
    a2 = np.round(arr2, rounding)

    _, ncols = a1.shape
    dtype = {
        'names':['f{}'.format(i) for i in range(ncols)], 
        'formats':ncols * [a1.dtype]
    }

    shared_coordinates = np.intersect1d(a1.view(dtype), a2.view(dtype))
    shared_coordinates = shared_coordinates.view(a1.dtype).reshape(-1, ncols)

    return shared_coordinates

a = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7.33231, 8.32321, 9.634522]
])

b = np.array([
    [1, 2, 3],
    [2, 1, 6],
    [7.3323, 8.32321, 9.634522]
])


shared_coordinates(a, b)

In [ ]:
def extract_neighbour_properties(self,
    freud_voronoi_query,
    voronoi_neighbours,
    index,
):
    """
    Extracts the properties of the neighbours of a given particle.
    """
    # Get the neighbour indices
    neighbour_indices = set(voronoi_neighbours[index])
    sites = [self[i] for i in neighbour_indices]
    
    # freud does not assing unique indices to each polytope vertex so we have
    # to do it ourselves.
    local_polytope = freud_voronoi_query.polytopes[index]
    
    for neighbour in neighbour_indices:
        neighbour_polytope = freud_voronoi_query.polytopes[neighbour]
        print(local_polytope)
        print(neighbour_polytope)
        print()
        shared_polytope_vertices = shared_coordinates(local_polytope, neighbour_polytope)
        #print(shared_polytope_vertices)

extract_neighbour_properties(zif8, voro, voronoi_neighbours, 0)

In [ ]:
ix = list(set(nlist_freud[0]))
print(ix)

all_vertices = voro.polytopes[0]
other_vertices = voro.polytopes[ix[0]]

a1 = np.round(all_vertices, 6)
a2 = np.round(other_vertices, 6)
nrows, ncols = a1.shape
dtype={'names':['f{}'.format(i) for i in range(ncols)],
       'formats':ncols * [a1.dtype]}
C = np.intersect1d(a1.view(dtype), a2.view(dtype))
C = C.view(a1.dtype).reshape(-1, ncols)

plt.scatter(*C[:, :2].T, c='g', s=100)
plt.plot(*np.vstack([points[0, :2], points[ix[0], :2]]).T)


plt.plot(*all_vertices[:, :2].T, color='blue')
plt.scatter(*points[0, :2], c='k', s=100)
plt.scatter(*points[ix, :2].T, c='r', s=50)

In [ ]:
np.vstack([points[0, :2], points[ix[0], :2]])

In [ ]:
a1 = np.round(all_vertices, 6)
a2 = np.round(other_vertices, 6)

nrows, ncols = a1.shape
dtype={'names':['f{}'.format(i) for i in range(ncols)],
       'formats':ncols * [a1.dtype]}

C = np.intersect1d(a1.view(dtype), a2.view(dtype))

# This last bit is optional if you're okay with "C" being a structured array...
C = C.view(a1.dtype).reshape(-1, ncols)
C

In [ ]:
C

In [ ]:
sites = [zif8[ix] for ix in set(voro.nlist[0])]
all_vertices = voro.polytopes[0]
center_coordinates = zif8[0].coords